<a href="https://colab.research.google.com/github/SEproject-Medihair/AI/blob/main/%ED%83%88%EB%AA%A8%EC%98%88%EC%B8%A1%EC%B5%9C%EC%A2%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

# 첫 번째 마운트: train 데이터 업로드
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [2]:
import shutil

# 데이터 디렉토리 설정
data_train_path = '/content/gdrive/My Drive/TrainFinal'
data_validation_path = '/content/gdrive/My Drive/valFinal'
data_test_path = '/content/gdrive/My Drive/valFinal'

destination_train_path = '/content/train_data'
destination_validation_path = '/content/validation_data'
destination_test_path = '/content/test_data'

# 데이터 복사
shutil.copytree(data_train_path, destination_train_path)
shutil.copytree(data_validation_path, destination_validation_path)
shutil.copytree(data_test_path, destination_test_path )


'/content/test_data'

In [3]:
!pip install efficientnet_pytorch

import time
import datetime
import os
import copy
import cv2
import random
import numpy as np
import json
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.optim import lr_scheduler
from torchvision import transforms, datasets
from torch.utils.data import Dataset,DataLoader
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
from PIL import Image
from efficientnet_pytorch import EfficientNet


  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=49d1337b7f5c73bb39825e50ec2dc99e26f0743e75adbb292a1736758240800d
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet_pytorch


In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
hyper_param_batch = 2

random_seed = 100
random.seed(random_seed)
torch.manual_seed(random_seed)

num_classes = 4
model_name = 'efficientnet-b7'

train_name = 'model6'

PATH = './scalp_weights/'

#data_train_path = './train_data/'+train_name+'/train'
#data_validation_path = './train_data/'+train_name+'/validation'
#data_test_path = './train_data/'+train_name+'/test'

image_size = EfficientNet.get_image_size(model_name)
print(image_size)

model = EfficientNet.from_pretrained(model_name, num_classes=num_classes)
model = model.to(device)


600


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b7-dcc49843.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b7-dcc49843.pth
100%|██████████| 254M/254M [00:03<00:00, 82.8MB/s]


Loaded pretrained weights for efficientnet-b7


In [5]:

transforms_train = transforms.Compose([
                                        transforms.Resize([int(600), int(600)], interpolation=4),
                                        transforms.RandomHorizontalFlip(p=0.5),
                                        transforms.RandomVerticalFlip(p=0.5),
                                        transforms.Lambda(lambda x: x.rotate(90)),
                                        transforms.RandomRotation(10),
                                        transforms.RandomAffine(0, shear=10, scale=(0.8, 1.2)),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                      ])

transforms_val = transforms.Compose([
                                        transforms.Resize([int(600), int(600)], interpolation=4),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                      ])


train_data_set = datasets.ImageFolder(data_train_path, transform=transforms_train)
val_data_set = datasets.ImageFolder(data_validation_path, transform=transforms_val)


dataloaders, batch_num = {}, {}

dataloaders['train'] = DataLoader(train_data_set,
                                    batch_size=hyper_param_batch,
                                    shuffle=True,
                                    num_workers=0)
dataloaders['val'] = DataLoader(val_data_set,
                                    batch_size=hyper_param_batch,
                                    shuffle=False,
                                    num_workers=0)

batch_num['train'], batch_num['val'] = len(train_data_set), len(val_data_set)

print('batch_size : %d,  train/val : %d / %d' % (hyper_param_batch, batch_num['train'], batch_num['val']))


batch_size : 2,  train/val : 771 / 383


In [6]:
import os

In [7]:
class_names = train_data_set.classes
print(class_names)

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    start_time = time.time()

    since = time.time()
    best_acc = 0.0
    best_model_wts = copy.deepcopy(model.state_dict())

    train_loss, train_acc, val_loss, val_acc = [], [], [], []

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        epoch_start = time.time()

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0
            num_cnt = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)

                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                num_cnt += len(labels)

            if phase == 'train':
                scheduler.step()

            epoch_loss = float(running_loss / num_cnt)
            epoch_acc  = float((running_corrects.double() / num_cnt).cpu()*100)

            if phase == 'train':
                train_loss.append(epoch_loss)
                train_acc.append(epoch_acc)
            else:
                val_loss.append(epoch_loss)
                val_acc.append(epoch_acc)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            if phase == 'val' and epoch_acc > best_acc:
                best_idx = epoch
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                print('==> best model saved - %d / %.1f'%(best_idx, best_acc))

            epoch_end = time.time() - epoch_start

            print('Training epochs {} in {:.0f}m {:.0f}s'.format(epoch, epoch_end // 60, epoch_end % 60))
            print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best valid Acc: %d - %.1f' %(best_idx, best_acc))

    model.load_state_dict(best_model_wts)
    save_path = PATH + 'aram_'+train_name+'.pt'

    # 디렉터리 생성
    os.makedirs(os.path.dirname(save_path), exist_ok=True)

    torch.save(model, PATH + 'aram_'+train_name+'.pt')
    torch.save(model.state_dict(), PATH + 'president_aram_'+train_name+'.pt')
    print('model saved')

    end_sec = time.time() - start_time
    end_times = str(datetime.timedelta(seconds=end_sec)).split('.')
    end_time = end_times[0]
    print("end time :", end_time)

    return model, best_idx, best_acc, train_loss, train_acc, val_loss, val_acc


criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.Adam(model.parameters(),lr = 1e-3, weight_decay=1e-4)
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

num_epochs = 1
train_model(model, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=num_epochs)

['0.양호', '1.경증', '2.중등도', '3.중증']
Epoch 0/0
----------
train Loss: 1.2507 Acc: 47.3411
Training epochs 0 in 70m 17s

val Loss: 2.2129 Acc: 42.8198
==> best model saved - 0 / 42.8
Training epochs 0 in 77m 58s

Training complete in 77m 58s
Best valid Acc: 0 - 42.8
model saved
end time : 1:17:59


(EfficientNet(
   (_conv_stem): Conv2dStaticSamePadding(
     3, 64, kernel_size=(3, 3), stride=(2, 2), bias=False
     (static_padding): ZeroPad2d((0, 1, 0, 1))
   )
   (_bn0): BatchNorm2d(64, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
   (_blocks): ModuleList(
     (0): MBConvBlock(
       (_depthwise_conv): Conv2dStaticSamePadding(
         64, 64, kernel_size=(3, 3), stride=[1, 1], groups=64, bias=False
         (static_padding): ZeroPad2d((1, 1, 1, 1))
       )
       (_bn1): BatchNorm2d(64, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
       (_se_reduce): Conv2dStaticSamePadding(
         64, 16, kernel_size=(1, 1), stride=(1, 1)
         (static_padding): Identity()
       )
       (_se_expand): Conv2dStaticSamePadding(
         16, 64, kernel_size=(1, 1), stride=(1, 1)
         (static_padding): Identity()
       )
       (_project_conv): Conv2dStaticSamePadding(
         64, 32, kernel_size=(1, 1), strid

In [8]:
from PIL import Image
import torchvision.transforms as transforms
import torch

from google.colab import files
uploaded = files.upload()

Saving 7701_A1LCAIHDH000085_1609665657006_5_RH.jpg to 7701_A1LCAIHDH000085_1609665657006_5_RH.jpg


In [9]:
# 전처리
def preprocess_image(image_path):
    transform = transforms.Compose([
    transforms.Resize([int(600), int(600)], interpolation=4),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.Lambda(lambda x: x.rotate(90)),
    transforms.RandomRotation(10),
    transforms.RandomAffine(0, shear=10, scale=(0.8, 1.2)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    image = Image.open(image_path).convert("RGB")
    input_tensor = transform(image)
    input_batch = input_tensor.unsqueeze(0)
    return input_batch

# 예측 수행
def predict_image(model, input_tensor):
    model.eval()
    with torch.no_grad():
        output = model(input_tensor)
    probabilities = torch.nn.functional.softmax(output[0], dim=0)
    predicted_class = torch.argmax(probabilities).item()
    return predicted_class, probabilities

# 이미지 경로 설정
image_path = "7701_A1LCAIHDH000085_1609665657006_5_RH.jpg"

# 전처리 및 예측
input_tensor = preprocess_image(image_path)
predicted_class, probabilities = predict_image(model, input_tensor)

# 결과 출력
print(f"Predicted Class: {predicted_class}")
print(f"Probabilities: {probabilities}")


Predicted Class: 2
Probabilities: tensor([0.0558, 0.1379, 0.7984, 0.0078])
